# 스파크 스트리밍 실습 3교시 : 스트리밍 모니터링 및 조회

> 스트리밍 애플리케이션을 모니터링 하는 기법을 익히고, 메모리 테이블을 생성하고, 노트북 상에서 활용할 수 있는 도구들을 통해서 좀 더 간편하게 디버깅 및 테스트 할 수 있습니다.

## 학습 목표
* 소켓 스트리밍 애플리케이션을 실행하고, 스트리밍 쿼리 모니터링을 실습합니다
  - `StreamingQueryProgress` 객체를 통한 모니터링 실습
  - `Dropwizard Metrics` 통한 Web UI 통한 모니터링 실습
* 메모리 테이블 싱크를 통해 테이블을 생성하고 Spark SQL 통한 조회를 실습합니다
  - JSON 파일을 소스로 하는 데이터 소스를 생성합니다
  - 집계결과를 메모리 싱크 테이블로 출력합니다
  - 셀 출력화면에 메모리 싱크 테이블 조회 결과를 출력하는 함수를 작성합니다
  - 결과 테이블을 `Spark SQL` 통하여 조회 합니다

## 목차
* [1. 스트리밍 쿼리 상태를 통한 모니터링 실습](#1.-스트리밍-쿼리-상태를-통한-모니터링-실습)
* [2. 메모리 싱크 테이블을 통한 조회 실습](#2.-메모리-싱크-테이블을-통한-조회-실습)


In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")

# 현재 기동된 스파크 애플리케이션의 포트를 확인하기 위해 스파크 정보를 출력합니다
spark

21/08/28 15:34:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).



## 1. 스트리밍 쿼리 상태를 통한 모니터링 실습



### 1.1 StreamingQuery 통하여 실행중인 쿼리 모니터링 하기

> '구조화된 스트리밍' 특성상, 실행중인 쿼리를 모니터링하는 기능은 아주 중요하며 다양한 방법을 통해 모니터링 할 수 있습니다

* 2교시에서 수행한 소켓 스트리밍 서버를 통한 예제 애플리케이션을 활용합니다
  - 아래의 명령으로 소켓 서버를 기동하고 스트리밍 애플리케이션을 기동합니다
```bash
# terminal
nc -lvp 9999 -n
```

In [2]:
# 소켓 서버로부터 단어를 가져와서 출력하는 가장 간단한 예제를 통해 모니터링 합니다

# step1: 데이터를 읽어올 스트림 리더를 생성합니다
wordCountHost = "localhost"
wordCountPort = 9999
wordCountReader = (
    spark
    .readStream
    .format("socket")
    .option("host", wordCountHost)
    .option("port", wordCountPort)
    .load()
)

# step2: 데이터로부터 단어의 수를 세는 카운터를 생성합니다
wordCounter = wordCountReader.select(explode(split(col("value"), "\s")).alias("word")).groupBy("Word").count().alias("Count")

# step3: 생성된 수치를 콘솔에 출력하는 출력을 생성합니다
queryName = "wordCount"
wordCountWriter = (
    wordCounter
    .writeStream
    .queryName(queryName) # 쿼리 테이블의 이름을 지정합니다
    .format("console") # 결과를 콘솔에 출력합니다
    .outputMode("complete") # 매번 전체 데이터를 내보냅니다
)

# step4: 얼마나 자주 수행될 지를 결정하는 트리거를 생성합니다
wordCountCheckpointDir = f"{work_dir}/tmp/{queryName}"
!rm -rf $wordCountCheckpointDir # 경우에 따라서 이미 존재하는 경로의 경우 오류가 발생할 수 있기 때문에 항상 제거합니다
wordCountTrigger = (
    wordCountWriter
    .trigger(processingTime="1 second")
    .option("checkpointLocation", wordCountCheckpointDir)
)

# step5: 해당 
wordCountQuery = wordCountTrigger.start()

21/08/28 15:34:21 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


> lastProgress 객체는 마지막의 상태를 저장하고 있으므로 마지막으로 수행한 애플리케이션의 상태를 확인하기 위한 용도로 사용됩니다.

* lastProgress 통해 StreamingQueryProgress(dictionary) 객체를 통해 확인합니다

| 컬럼 | 설명 |
| --- | --- |
| id | Unique Identifier - 체크포인트 위치와 1:1 매칭되는 유일한 구분자로, 체크포인트 경로가 삭제되기 전까지는 동일한 값이 유지됩니다 |
| runId | Unique Identifier - 현재 (지)시작된 쿼리 인스턴스를 가리키는 구분자이며, 매 실행시마다 변경됩니다 |
| numInputRows | 마지막 '마이크로 배치' 작업에 수행 했던 입력 로우의 수 |
| inputRowsPerSecond | 데이터 소스로부터 입력 로우 수를 말하며, 마지막 수행된 '마이크로 배치' 평균 소요시간을 기준으로 계산 됩니다 |
| processedRowsPerSecond | 데이터 싱크로 처리되어 저장되는 로우의 비율을 말합니다. `입력 로우의 수 대비 처리하는 로우의 수가 일정하게 낮다면` 지연되고 있다고 말할 수 있습니다 | 
| sources and sink | 데이터 소스와 싱크에 대한 정보 |

> 터미널 화면에서 임의의 문자열을 타이핑 하고, 결과를 터미널에서 확인한 이후에 마지막 상태를 확인합니다

```bash
notebook     | -------------------------------------------
notebook     | Batch: 1
notebook     | -------------------------------------------
notebook     | +-----+-----+
notebook     | | Word|count|
notebook     | +-----+-----+
notebook     | |world|    1|
notebook     | |hello|    1|
notebook     | +-----+-----+
```

In [21]:
JSON(wordCountQuery.lastProgress)

<IPython.core.display.JSON object>

In [23]:
wordCountQuery.stop()

<br>

### 반복적으로 상태를 확인 하는 displayStatus 함수를 생성합니다

```python
# 스트림 쿼리의 상태를 주기적으로 조회하는 함수 (name: 이름, query: Streaming Query, iterations: 반복횟수, sleep_secs: 인터벌)
def displayStatus(name, query, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)      # Output Cell 의 내용을 지웁니다
        display('[' + name + '] Iteration: '+str(i)+', Status: '+query.status['message'])
        display(query.lastProgress)  # 마지막 수행된 쿼리의 상태를 출력합니다
        sleep(sleep_secs)            # 지정된 시간(초)을 대기합니다
        i += 1
```

In [9]:
def displayStatus(name, query, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)
        display('[' + name + '] Iteration: '+str(i)+', Status: '+query.status['message'])
        display(query.lastProgress)
        sleep(sleep_secs)    
        i += 1

#### 터미널 환경에서 넷캣(nc) 명령으로 소켓 서버를 띄웁니다

```bash
nc -lvp 9999 -n
Listening on 0.0.0.0 9999
```

> 서버를 기동하고 Listening 메시지가 뜨면 정상입니다

In [101]:
!rm -rf $wordCountCheckpointDir
wordCountQuery = wordCountTrigger.start()
displayStatus("status of query", wordCountQuery, 40, 3)
wordCountQuery.stop()

'[status of query] Iteration: 60, Status: Waiting for next trigger'

{'id': '9ea980c7-15f8-404d-92b8-5351dac7140b',
 'runId': '2f76de05-aba0-4d5a-a97a-7343f3502dd5',
 'name': 'wordCount',
 'timestamp': '2021-08-15T16:27:16.001Z',
 'batchId': 5,
 'numInputRows': 0,
 'inputRowsPerSecond': 0.0,
 'processedRowsPerSecond': 0.0,
 'durationMs': {'latestOffset': 0, 'triggerExecution': 0},
 'stateOperators': [{'numRowsTotal': 4,
   'numRowsUpdated': 0,
   'memoryUsedBytes': 3008,
   'customMetrics': {'loadedMapCacheHitCount': 15,
    'loadedMapCacheMissCount': 5,
    'stateOnCurrentVersionSizeBytes': 1176}}],
 'sources': [{'description': 'TextSocketV2[host: localhost, port: 9999]',
   'startOffset': 0,
   'endOffset': 0,
   'numInputRows': 0,
   'inputRowsPerSecond': 0.0,
   'processedRowsPerSecond': 0.0}],
 'sink': {'description': 'org.apache.spark.sql.execution.streaming.ConsoleTable$@35cbdff9',
  'numOutputRows': 0}}

<br>

### 1.2 스파크 Web UI 통한 모니터링 하기

* 소켓 서버를 통해 작업하기 보다는 파일을 통해 천천히 스트리밍 처리를 하면서 모니터링을 하기 위해 파일을 기반으로 작성합니다

#### 활동 로그를 읽어서 주기적으로 스트리밍 처리를 하여 `custom_count` 테이블로 저장합니다

In [17]:
customSchema = (
    StructType()
    .add(StructField("Arrival_Time", LongType()))
    .add(StructField("Creation_Time", LongType()))
    .add(StructField("Device", StringType()))
    .add(StructField("Index", LongType()))
    .add(StructField("Model", StringType()))
    .add(StructField("User", StringType()))
    .add(StructField("gt", StringType()))
    .add(StructField("x", DoubleType()))
    .add(StructField("y", DoubleType()))
    .add(StructField("z", DoubleType()))
)
activityPath = f"{work_data}/activity-data"
customReader = (
    spark
    .readStream
    .format("json")
    .schema(customSchema)
    .option("maxFilesPerTrigger", 1)
    .load(activityPath)
)
customCounter = (
    customReader.groupBy("gt").count()
)
queryName = "custom_count"
customWriter = (
    customCounter
    .writeStream
    .queryName(queryName)
    .format("console")
    .outputMode("update")
)
checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation
customTrigger = (
    customWriter
    .trigger(processingTime="1 second")
    .option("checkpointLocation", checkpointLocation)
)

In [18]:
!rm -rf $checkpointLocation
customQuery = customTrigger.start()
customQuery.awaitTermination(80)
customQuery.stop()

In [109]:
spark.streams.active


> `http://vm<number>.aiffel.co.kr:4040/jobs` 페이지에 접속하여, 현재 실행 중인 `custom_count` 정보를 확인합니다 (여러개의 SparkContext 가 존재하는 경우 포트가 4041, 4042 로 늘어날 수 있습니다)

#### 웹 UI 통한 디버깅
  - http://localhost:4040/jobs
![spark-streaming-ui](images/spark-streaming-ui.png)

#### 개별 스트리밍 쿼리의 상태
![spark-streaming-stats](images/spark-streaming-stats.png)

<br>

### 1.3 StreamingQueryListeners 객체를 이용하여 매트릭 게시하기

> StreamingQueryListener 인터페이스를 통해 다양한 이벤트를 체크할 수 있습니다. 단, `인터페이스 구현을 통한 컴파일 언어만 지원하기 때문에 pyspark 에서는 사용할 수 없습`니다.

* 아래의 예제는 3가지 이벤트(시작, 종료, 진행 등)를 모니터링 하는 리스너를 구현합니다
```scala
import org.apache.spark.sql.streaming._
val myListener = new StreamingQueryListener() {
    override def onQueryStarted(event: QueryStartedEvent): Unit = {
        println("Query started: " + event.id)
    }
    override def onQueryTerminated(event: QueryTerminatedEvent): Unit = {
        println("Query terminated: " + event.id)
    }
    override def onQueryProgress(event: QueryProgressEvent): Unit = {
        println("Query made progress: " + event.progress)
    }
}
```

* 구현된 리스너를 SparkSession 실행 전에 등록합니다
```scala
    spark.streams.addListener(myListener)
```

### <font color=blue>1. [중급]</font> 소켓 서버 예제를 `SocketCount` 쿼리라는 이름으로 코딩하고,  Web UI 통하여 상태를 확인하세요

> 기존에 수행되고 있는 동일한 애플리케이션 이름이 있다면 실행되지 않으므로 Web UI 에서 확인 후 실행합니다

* 소켓 서버 애플리케이션을 기동합니다 (이번 장의 처음 예제 코드를 그대로 사용합니다)
  - 호스트 : localhost
  - 포트 : 9999
* 변환 작업
  - 소켓으로 전달 받은 공백으로 구분된 문자열을 단어로 쪼개어(split, explode), "Word", "Count" 컬럼으로 alias 합니다
* 콘솔 싱크
  - 쿼리 : SocketCount
  - 포맷 : console
  - 모드 : complete
* 트리거링
  - 1초에 한 번 트리거링
  - 체크포인트 : /home/jovyan/work/lgde-spark-stream/tmp/wordCount
* 애플리케이션
  - 타임아웃 : 3분 내외 (소켓 서버 테스트 할 시간)
  - 테스트 후 애플리케이션을 종료해 주세요

<details><summary>[정답] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었고, Web UI 에서 `SocketCount` 쿼리가 보인다면 성공입니다

```python
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
queryName = "SocketCount"
socketReader = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()
socketCounter = socketReader.select(explode(split(col("value"), "\s")).alias("word")).groupBy("Word").count().alias("Count")
socketWriter = socketCounter.writeStream.queryName(queryName).format("console").outputMode("complete")
wordCountCheckpointDir = f"{work_dir}/tmp/{queryName}"
!rm -rf $wordCountCheckpointDir
socketTrigger = socketWriter.trigger(processingTime="1 second").option("checkpointLocation", wordCountCheckpointDir)
socketQuery = socketTrigger.start()
socketQuery.awaitTermination(180)
socketQuery.stop()
```

</details>

In [25]:
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


<br>

## 2. 메모리 싱크 테이블을 통한 조회 실습

> 라이브 스트리밍 데이터를 통해 실습 혹은 개발을 하기에는 시뮬레이션이 어렵거나, 원하는 대로 테스트하기 어려운 경우가 많습니다. 이러한 경우 원본 데이터를 파일로 저장하고, 출력을 memory 테이블로 설정하여 임의의 스트리밍 데이터를 실습하고 조회할 수 있습니다

### 2.1 JSON 스트리밍을 통해 메모리 싱크 테이블 생성

In [41]:
# 파일을 읽어서 스키마를 확인하고
flightPath = f"{work_data}/flight-data/json"
flightJson = spark.read.json(flightPath)
flightJson.printSchema()

# 커스텀 스키마를 생성하고
flightSchema = (
    StructType()
    .add(StructField("DEST_COUNTRY_NAME", StringType()))
    .add(StructField("ORIGIN_COUNTRY_NAME", StringType()))
    .add(StructField("count", LongType()))
)
print(flightSchema)

# 데이터 소스를 스트림으로 읽어서
flightReader = (
    spark
    .readStream
    .format("json")
    .schema(flightSchema)
    .option("maxFilesPerTrigger", 1)
    .load(flightPath)
)

# 변환 로직을 계산하고
flightCounter = flightReader.groupBy("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").agg(sum("count").alias("count"))

# 싱크 테이블을 메모리로 설정하고
queryName = "memory_flight"
flightWriter = (
    flightCounter
    .writeStream
    .queryName(queryName)
    .format("memory")
    .outputMode("update")
)

# 트리거 설정을 하고
!rm -rf $flightCheckpointLocation
flightCheckpointLocation = f"{work_dir}/tmp/{queryName}"
flightTrigger = (
    flightWriter
    .trigger(processingTime = "1 second")
    .option("checkpointLocation", flightCheckpointLocation)
)

# 애플리케이션을 기동합니다
flightQuery = flightTrigger.start()
flightQuery.awaitTermination(10)
flightQuery.stop()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))


In [45]:
# 애플리케이션은 종료되어도 해당 가상 테이블은 여전히 존재하므로, 확인이 가능합니다
memory_flight = spark.sql(f"select * from {queryName}")
memory_flight.orderBy(desc("count")).limit(5)

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,United States,2119795
United States,United States,1761441
United States,United States,1418309
United States,United States,1070857
United States,United States,700855


<br>

### 2.2 스트리밍 테이블 조회를 위한 모니터링 함수 작성

> 스트리밍 테이블을 주기적으로 조회하는 함수를 생성하여 모니터링 할 수 있습니다

#### 2.2.1 쿼리 메소드
* activityQuery.explain() : 쿼리의 실행계획을 출력
* activityQuery.stop() : 쿼리를 중지합니다
* activityQuery.awaitTermination() # 쿼리가 종료(query.stop() or exception)될 때까지 대기합니다

#### 2.2.2 쿼리 속성
* activityQuery.id : 실행되는 쿼리의 고유식별자 (체크포인트로부터 재시작 되어도 변하지 않음)
* activityQuery.runId : 실행중인 쿼리의 고유 식별자 (시작 및 재시작 시에 변경)
* activityQuery.name : 자동으로 생성된 혹은 이용자가 명시한 쿼리의 이름 - queryName("activity_counts")
* activityQuery.exception : 오류와 함께 종료된 쿼리의 예외 정보
* activityQuery.recentProgress : 가장 최근의 쿼리가 수행한 상태를 담고 있는 배열 [StreamingQueryProgress]
* activityQuery.lastProgress : 마지막으로 수행한 쿼리의 상태 StreamingQueryProgress

In [46]:
# 스트림 테이블을 주기적으로 조회하는 함수 (name: 이름, sql: Spark SQL, iterations: 반복횟수, sleep_secs: 인터벌)
def displayStream(name, sql, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)              # 출력 Cell 을 지웁니다
        display('[' + name + '] Iteration: '+str(i)+', Query: '+sql)
        spark.sql(sql).show(truncate=False)  # Spark SQL 을 수행합니다
        sleep(sleep_secs)                    # sleep_secs 초 만큼 대기합니다
        i += 1

In [47]:
displayStream("count_of_flight", f"select * from {queryName}", 5, 3)

'[count_of_flight] Iteration: 10, Query: select * from memory_flight'

+------------------------+----------------------+-----+
|DEST_COUNTRY_NAME       |ORIGIN_COUNTRY_NAME   |count|
+------------------------+----------------------+-----+
|United States           |India                 |69   |
|Costa Rica              |United States         |477  |
|Turks and Caicos Islands|United States         |136  |
|United States           |Afghanistan           |2    |
|United States           |Netherlands           |570  |
|Iceland                 |United States         |118  |
|Marshall Islands        |United States         |77   |
|Luxembourg              |United States         |91   |
|El Salvador             |United States         |519  |
|Samoa                   |United States         |28   |
|Sint Maarten            |United States         |61   |
|Hong Kong               |United States         |252  |
|Suriname                |United States         |12   |
|United States           |Bosnia and Herzegovina|1    |
|United States           |Portugal              

### <font color=blue>2. [중급]</font> f"{work_data}/activity-data" 경로의 JSON 파일을 읽고, displayStream 함수를 이용해 모니터링 하세요

* 스트림 소스를 이용하여 스트리밍 애플리케이션을 작성합니다
  - 소스 : /home/jovyan/work/data/activity-data
  - 포맷 : json
  - 원본 데이터 파일의 스키마를 그대로 활용합니다
* 변환 작업
  - 핸드폰 사용 패턴을 나타내는 컬럼('gt')를 기준으로 빈도수('count')를 출력하는 스트리밍 애플리케이션을 구현합니다
* 콘솔 싱크
  - 쿼리 : memory_activity
  - 포맷 : memory
  - 모드 : complete
* 트리거링
  - 1초에 한 번 트리거링
  - 체크포인트 : /home/jovyan/work/lgde-spark-stream/tmp/memory_activity
* 애플리케이션
  - 타임아웃 : 2분 내외 (모든 파일을 읽고 처리할 충분한 시간)
  - 테스트 후 애플리케이션을 종료해 주세요
* 모니터링
  - 조회 : 사용패턴 별 빈도수 (gt, count)
  - 정렬 : 빈도수 역순 (count desc)

<details><summary>[정답] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었고, Web UI 에서 `SocketCount` 쿼리가 보인다면 성공입니다

```python
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
activityPath = f"{work_data}/activity-data"
activityJson = spark.read.json(activityPath)
activityReader = spark.readStream.schema(activityJson.schema).option("maxFilesPerTrigger", 1).json(activityPath)
activityCounter = activityReader.groupBy("gt").count()
queryName = "memory_activity"
activityWriter = activityCounter.writeStream.queryName(queryName).format("memory").outputMode("complete")
checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation
activityTrigger = activityWriter.trigger(processingTime="1 second").option("checkpointLocation", checkpointLocation)
activityQuery = activityTrigger.start()
displayStream("count_of_activity", f"select * from {queryName} order by count desc", 30, 3)
activityQuery.stop()
```

</details>

In [49]:
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


'[count_of_activity] Iteration: 30, Query: select * from memory_activity order by count desc'

+----------+-------+
|gt        |count  |
+----------+-------+
|walk      |1060402|
|sit       |984714 |
|stand     |910783 |
|bike      |863710 |
|stairsup  |836598 |
|null      |835725 |
|stairsdown|749059 |
+----------+-------+



### <font color=blue>3. [중급]</font> 스파크 코어 라이브러리를 이용하여 2번 과제의 결과와 일치하는지 여부를 확인하는 코드를 작성하세요

* 데이터 소스
  - 위치 : /home/jovyan/work/data/activity-data
  - 포맷 : json
* 데이터 변환
  - 조건 : 사용패턴 컬럼(gt) 기준으로 그룹(groupBy)한 빈도(count)

<details><summary>[정답] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
activityPath = f"{work_data}/activity-data"
activityJson = spark.read.json(activityPath)
activityJson.groupBy("gt").count().alias("count").orderBy(desc("count"))
```

</details>


In [51]:
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)


gt,count
walk,1060402
sit,984714
stand,910783
bike,863710
stairsup,836598
null,835725
stairsdown,749059
